https://www.jianshu.com/p/7c08e9d79514

教程网址

# 数据处理

## 读取数据

In [1]:
import numpy as np
import pandas as pd
data=pd.read_csv('demo-customer-churn-ann.csv')

In [2]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


+ RowNumber：行号，这个肯定没用，删除
+ CustomerID：用户编号，这个是顺序发放的，删除
+ Surname：用户姓名，对流失没有影响，删除
+ CreditScore：信用分数，这个很重要，保留
+ Geography：用户所在国家/地区，这个有影响，保留
+ Gender：用户性别，可能有影响，保留
+ Age：年龄，影响很大，年轻人更容易切换银行，保留
+ Tenure：当了本银行多少年用户，很重要，保留
+ Balance：存贷款情况，很重要，保留
+ NumOfProducts：使用产品数量，很重要，保留
+ HasCrCard：是否有本行信用卡，很重要，保留
+ IsActiveMember：是否活跃用户，很重要，保留
+ EstimatedSalary：估计收入，很重要，保留
+ Exited：是否已流失，这将作为我们的标签数据


> 上述数据列甄别过程，就叫做“特征工程”（Feature Engineering），这是机器学习里面最常用的数据预处理方法。如果我们的数据量足够大，机器学习模型足够复杂，是可以跳过这一步的。但是由于我们的数据只有10000条，还需要手动筛选特征

In [3]:
Y=data.Exited

In [4]:
data.drop(labels=['RowNumber','CustomerId','Surname','Exited'],axis=1,inplace=True)  #删除不需要的数据列，只保留需要的

In [5]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


Y.head()

## 字符串变数字

In [6]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder 
#labelEncoder可以将类别信息变成数值——就是把字符串变成数值

In [7]:
#因为要转换性别和地理位置两列，所以建立两个不同的labelcoder
GeoLabelCoder=LabelEncoder()
data.Geography=GeoLabelCoder.fit_transform(data.Geography)
GenderLabelCoder=LabelEncoder()
data.Gender=GenderLabelCoder.fit_transform(data.Gender)

In [8]:
data.head()  #看了一下，已经变成数值了， 类别信息变成数值， 用数字表示类别

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [9]:
data.Geography.value_counts()

0    5014
1    2509
2    2477
Name: Geography, dtype: int64

但是这里有一个问题，性别是0 1,0 表示女 1 表示男，两种取值只是描述类别不同，没有其他别的意思<br>
但是地理位置取值有三种，0 法国，1德国 2西班牙，但是这三者之间并没有序列（大小）关系<br>
**我们只是想用数字描述类别，但是取值有数量的差异，就会给机器带来歧义，它并不清楚不同的取值只是某个国家的代码，可能会把这种大小关系代入模型计算，从而产生错误的结果**<br>
**为了解决问题，使用one_hot编码，把类别的数值编程多个变量组合表示**

以前NLP中，把词→数字，那个最后会反映到句子向量中，单个词对应的数字并不具有大小关系



## One_hot编码变成标称特征

### 特征矩阵

In [10]:
# 这个任务里，只需要转换国别这列，玉树芝兰的作者是这样写的
onehotencoder=OneHotEncoder(categorical_features=[1]) #Geograph是第一列，categorical_features写位置信息
data=onehotencoder.fit_transform(data).toarray()

在处理真实的数据集的时候，我们经常会遇见一个或多个的类别数据的特征。类别数据可以被分为标称特征(nominal feature)和有序特征(ordinal feature)。<br>有序特征指的是类别的值是有序的或者是可以排序的，例如，衣服的尺码S、M、X、XL、XXL...就是属于有序特征
<br>再例如，衣服的颜色，黑色、蓝色、白色、黄色...这些就属于标称特征。


In [11]:
data[0]

array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 6.1900000e+02,
       0.0000000e+00, 4.2000000e+01, 2.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0134888e+05])

**这样之后特征矩阵DataFrame就被转换成了一个数组narray，numpy中的数据类型。PS:所有被OnehotEnder转换的列会排在最前面，然后才是那些保持原样的数据列**

比如第一个array，前三个数字其实是1 0 0 ，表示国别，然后是619，……，除了前三个数，还有10个数，刚好

 https://blog.csdn.net/sinat_29957455/article/details/79452141
 这个博客也是这么写的

**到这里，也还没有完，因为很明显，三个国家的编码，法国从0变成了1 0 0 ，德国从1变成了 0 1 0 西班牙从2 变成了 0 0 1**<br>
虽然这样 不会出现0和1之外的数字来描述类别，机器就不会产生误会，错把类别当成大小来计算<br>
但是由于本例中，OnehotEncoder转换出来的三列数字，实际上是不独立的，给定其中两列的信息，就可以推出第三列取值<br>
好比某一行前两列数字是(0,0),那么第三列肯定是1，这是由转换规则决定的<br>
如果你做过多元线性回归，应该知道这种情况下，我们需要去掉其中一列，才能继续分析，不然会落入“虚拟变量陷阱”（dummy variable trap）

所以我们删掉第0列，避免掉坑里

PS：三个数字，用二进制表示，其实也只用两位数啊，00,01,10,11 。表示三种类别绰绰有余

In [12]:
data=np.delete(data,[0],1) 
#arr, obj, axis=None 参数说明，第一个参数是要进行删除操作的array，第二个是要删除的对象，一般用索引，
#第三个数是轴，1，列方向，0 行方向

In [13]:
data[0]

array([0.0000000e+00, 0.0000000e+00, 6.1900000e+02, 0.0000000e+00,
       4.2000000e+01, 2.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0134888e+05])

到此为止，特征矩阵基本完成。

In [15]:
data[1] #在对国别进行one-hot转码的时候，就捎带把data从df变成了一个numpy矩阵，每行是一个array，代表df中的一条记录

array([0.0000000e+00, 1.0000000e+00, 6.0800000e+02, 0.0000000e+00,
       4.1000000e+01, 1.0000000e+00, 8.3807860e+04, 1.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 1.1254258e+05])

### 标签数据处理

In [16]:
Y=Y[:,np.newaxis] #之前的Y是DataFrame类型的，是一列，现在要转换为向量，一般是行向量，现在就变成array了
#上面的代码实质就是将原本的(10,)移到行上，然后新增一列
#https://blog.csdn.net/molu_chase/article/details/78619731
# 这个博客写的很清楚，简单来说： 
#假设y本来是一个1*10的DataFrame，如果直接用array，其shape就变成（10，），想变成1*10这样标准的格式，需要加一个newaxis
# 简单来说，： 代表这个行就是Y的数据，然后放在新的一列上， 但是由于只有一行数据，还是用[]括着的，所以只有一列

In [17]:
Y

array([[1],
       [0],
       [1],
       ...,
       [1],
       [1],
       [0]], dtype=int64)

这样，在后面训练的时候，就可以和前面的特征矩阵一一对应来操作计算了

同时，既然标签代表类别，也把它用onehot转换，方便后面做分类学习

In [18]:
Yonehot=OneHotEncoder()
Y=Yonehot.fit_transform(Y).toarray()

In [19]:
Y

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])

现在标签变成了两列，第一列是顾客存留，第二列是顾客流失， 当用单纯的0，1数字表示的时候，1表示流失，0表示没有<br>
用onehot编码之后，变成了两列，第二列的数字其实和以前一样，第一列数字其实是第一列取反，所以第二列=以前 表示 是否流失  所以第一列 就表示 存留

## 划分测试集训练集

In [20]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data,Y,test_size=0.2,random_state=0)

D:\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [21]:
len(x_train)

8000

## 数据标准化

**由于此时data已经变成了array，不再是DataFrame，所以不能使用describe来查看每列数据的统计特征**

In [22]:
databackup=pd.read_csv('demo-customer-churn-ann.csv')

In [23]:
databackup.drop(labels=['RowNumber','CustomerId','Surname','Exited'],axis=1,inplace=True)

In [24]:
databackup.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000


**观察可以看到，std值差的很多，由于原先每一列数字的取值范围各不相同，所有有的列的方差远远大于其他列，这会对机器造成困扰**

describe只显示数值列的结果

>数据的标准化处理，可以在保持列内数据多样性的同时，尽量减少不同类别之间差异的影响，让机器公平的对待全部特征

In [23]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test) #上一步fit过了，这步就只需要transform了

> 注意，只对特征矩阵进行标准化，标签是不能动（本来就没必要动）。同时训练集测试集按照统一的标准变化，所以在对训练集操作的时候，用了fit_transform，先拟合后转换，而测试集上，直接用训练集拟合的结果，只转换

In [24]:
x_train[:3]

array([[-0.5698444 ,  1.74309049,  0.16958176, -1.09168714, -0.46460796,
         0.00666099, -1.21571749,  0.8095029 ,  0.64259497, -1.03227043,
         1.10643166],
       [ 1.75486502, -0.57369368, -2.30455945,  0.91601335,  0.30102557,
        -1.37744033, -0.00631193, -0.92159124,  0.64259497,  0.9687384 ,
        -0.74866447],
       [-0.5698444 , -0.57369368, -1.19119591, -1.09168714, -0.94312892,
        -1.031415  ,  0.57993469, -0.92159124,  0.64259497, -1.03227043,
         1.48533467]])

In [25]:
databackup.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [26]:
databackup_after=pd.DataFrame(x_train,columns=['Geography1','Geography2','CreditScore','Gender','age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary'])

In [27]:
databackup_after.head()

,Geography1,Geography2,CreditScore,Gender,age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.569844,1.743090,0.169582,-1.091687,-0.464608,0.006661,-1.215717,0.809503,0.642595,-1.032270,1.106432
1,1.754865,-0.573694,-2.304559,0.916013,0.301026,-1.377440,-0.006312,-0.921591,0.642595,0.968738,-0.748664
2,-0.569844,-0.573694,-1.191196,-1.091687,-0.943129,-1.031415,0.579935,-0.921591,0.642595,-1.032270,1.485335
3,-0.569844,1.743090,0.035566,0.916013,0.109617,0.006661,0.473128,-0.921591,0.642595,-1.032270,1.276528
4,-0.569844,1.743090,2.056114,-1.091687,1.736588,1.044737,0.810193,0.809503,0.642595,0.968738,0.558378


这样转换成DataFrame是为了方便观察，前两列是 国别转换后的结果，从0 1 变成了 -0.569844	1.743090	-0.573694 但是反正只有这三种数字

In [28]:
databackup_after.describe()

,Geography1,Geography2,CreditScore,Gender,age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
count,8.000000e+03,8.000000e+03,8.000000e+03,8.000000e+03,8.000000e+03,8.000000e+03,8.000000e+03,8.000000e+03,8.000000e+03,8.000000e+03,8.000000e+03
mean,3.631817e-16,1.090655e-16,4.474476e-16,5.285217e-16,1.474237e-16,7.366330e-17,-2.335077e-16,-3.251843e-16,-1.310341e-16,-2.786660e-17,6.837170e-16
std,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00
min,-5.698444e-01,-5.736937e-01,-3.098346e+00,-1.091687e+00,-1.995875e+00,-1.723466e+00,-1.215717e+00,-9.215912e-01,-1.556190e+00,-1.032270e+00,-1.740871e+00
25%,-5.698444e-01,-5.736937e-01,-6.963677e-01,-1.091687e+00,-6.560163e-01,-1.031415e+00,-1.215717e+00,-9.215912e-01,-1.556190e+00,-1.032270e+00,-8.499475e-01
50%,-5.698444e-01,-5.736937e-01,1.494794e-02,9.160134e-01,-1.774954e-01,6.660988e-03,3.322869e-01,-9.215912e-01,6.425950e-01,9.687384e-01,2.564660e-03
75%,-5.698444e-01,-5.736937e-01,6.953368e-01,9.160134e-01,4.924340e-01,6.987116e-01,8.224106e-01,8.095029e-01,6.425950e-01,9.687384e-01,8.594117e-01
max,1.754865e+00,1.743090e+00,2.056114e+00,9.160134e-01,4.703418e+00,1.736788e+00,2.593763e+00,4.271691e+00,6.425950e-01,9.687384e-01,1.734953e+00


主要是为了观察std，这样整体数据的方差就小了很多，这样机器学习起来，就会更公平，不至于有很大的偏见

至此，数据清理和转换工作完成

# 决策树

之前用过决策树，其实这个问题和之前的 https://www.jianshu.com/p/67a71e366516 决策树决定贷款很像

In [29]:
from sklearn import tree
clf=tree.DecisionTreeClassifier()
clf=clf.fit(x_train,y_train)

In [30]:
y_pred=clf.predict(x_test)

In [31]:
from sklearn import metrics

In [32]:
print(metrics.classification_report(y_pred,y_test))

             precision    recall  f1-score   support

          0       0.87      0.89      0.88      1558
          1       0.57      0.52      0.55       442

avg / total       0.80      0.81      0.81      2000



In [33]:
metrics.accuracy_score(y_pred,y_test)

0.8085

# 深度学习

深度学习的使用场景，往往是因为原有的 经典 机器学习模型过于简单，无法把握 数据的复杂特性

http://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=2,2&seed=0.83954&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false

这个网址，是TensorFlow游乐场？ 总之是一个比较趣味的东西，需要再谷歌访问助手打开的情况下才可以访问，不然无法访问

上面那个网址很好玩，可以根据 玉树芝兰的 指引 去看看，因为设置比较简单，所以可能要多等一会，那个epoches有时候要到1k才会看到明显的颜色效果

许多时候模型过于简单带来的问题，可以通过加深隐藏层次，增加神经元的方法提升模型复杂度，加以改进

目前流行的划分方法，是用隐藏层的数量来区分是否“深度”，当神经网络中隐藏层数达到三层以上时，就被称为“深度神经网络”或者“深度学习”

> 在TensorFlow上高度抽象的框架有keras，TensorLayer以及tflearn，知乎上有个这三者比较的话题，反正就是keras不好。。。所以这里使用tflearn

## 建模

In [34]:
import tflearn

In [35]:
net=tflearn.input_data(shape=[None,11]) #因为特征矩阵共有11列，所以列是11。这里是网络的输入层。

另外，None这里代表的是要输入的特征矩阵行数，因为现在是搭建模型，后面我们可能会将特征矩阵一次输入，也可能分块输入，大小不确定，所以用None来暂替

In [36]:
net=tflearn.fully_connected(net,6,activation='relu') #隐藏层是三个全连接层，每层6个神经元，激活函数是relu，取非负数字
net=tflearn.fully_connected(net,6,activation='relu')
net=tflearn.fully_connected(net,6,activation='relu')

因为至今不存在最优神经元数量的计算公式。工程界的一种做法是： 把输入层的神经元数量，加上输出层神经元数量，除以2取整。

这篇教程也是这样做的，得出6个

输入的神经元应该是11，输出是2，所以相加除以2取整=6

In [37]:
net=tflearn.fully_connected(net,2,activation='softmax')
net=tflearn.regression(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


输出层用两个神经元做输出，并且显式的说明了使用回归方法。输出层的激活函数是softmax，因为在处理分类任务的时候，softmax比较合适，会告诉我们每一类的可能性，其中数值最高的一类，可以作为分类结果

In [40]:
model=tflearn.DNN(net,tensorboard_verbose=3,tensorboard_dir='E:/jupter/tmp/tflearn') #告诉tflearn,使用刚刚的net的结构来生成模型

https://github.com/tflearn/tflearn/issues/701

感谢这个人的问题，让我看到了代码中原来可以控制log的目录，使用这个目录进行tensorboard的使用，就不会报错说 没有可用的数据

tensorboard --logdir='E:/jupter/tmp/tflearn'

In [41]:
model.fit(x_train,y_train,n_epoch=30,batch_size=32,show_metric=True) #有了模型，就可以拟合，和scikit-learn很像

Training Step: 7499  | total loss: 0.38062 | time: 1.083s
| Adam | epoch: 030 | loss: 0.38062 - acc: 0.8538 -- iter: 7968/8000
Training Step: 7500  | total loss: 0.37905 | time: 1.087s
| Adam | epoch: 030 | loss: 0.37905 - acc: 0.8559 -- iter: 8000/8000
--


很明显，默认使用Adam优化，最后结果损失是0.38，准确率0.84

## 评估

In [42]:
x_test[0]

array([ 1.75486502, -0.57369368, -0.55204276, -1.09168714, -0.36890377,
        1.04473698,  0.8793029 , -0.92159124,  0.64259497,  0.9687384 ,
        1.61085707])

In [44]:
y_pred=model.predict(x_test)

In [46]:
y_pred[0]

array([0.7100878 , 0.28991222], dtype=float32)

所以第0个顾客不流失的概率是0.71，流失的概率是0.28

In [47]:
y_test[0]

array([1., 0.])

和实际的结果一致

In [49]:
score=model.evaluate(x_test,y_test)

In [50]:
score

[0.8529999995231629]

测试集上的准确率比训练集还高。。。